<a href="https://www.kaggle.com/code/singhayush16/titanic-survival-1?scriptVersionId=144081449" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer,OneHotEncoder,LabelEncoder,PowerTransformer
from sklearn.pipeline import Pipeline

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score,cross_val_predict

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
train=pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test=pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train.HomePlanet.unique()

In [ ]:
train.CryoSleep.unique()

In [ ]:
train.Destination.unique()

In [ ]:
train.VIP.unique()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
train['Cabin'].fillna('//',inplace=True)

In [ ]:
train[['deck','num','side']]=train['Cabin'].str.split('/',expand=True)

In [ ]:
train[['deck','num','side']]=train[['deck','num','side']].replace('',np.NaN)

In [ ]:
train['num']=train['num'].astype(float)

In [ ]:
test['Cabin'].fillna('//',inplace=True)
test[['deck','num','side']]=test['Cabin'].str.split('/',expand=True)
test[['deck','num','side']]=test[['deck','num','side']].replace('',np.NaN)
test['num']=test['num'].astype(float)

In [ ]:
train.info()

In [ ]:
for i,el in enumerate(train.columns):
    print(i,el)

In [ ]:
X=train.drop(['PassengerId','Transported','Name','Cabin'],axis=1)
y=train['Transported']

In [ ]:
X_train,X_valid,y_train,y_valid=train_test_split(X,y,train_size=0.8,random_state=42)

In [ ]:
X_train.shape,X_valid.shape,y_train.shape,y_valid.shape

In [ ]:
X.head()

In [ ]:
le=LabelEncoder()
y_train=le.fit_transform(y_train)
y_valid=le.transform(y_valid)

In [ ]:
trf1=ColumnTransformer([
    ('impute_home',SimpleImputer(strategy='most_frequent'),[0]),
    ('impute_sleep',SimpleImputer(strategy='most_frequent'),[1]),
    ('impute_dest',SimpleImputer(strategy='most_frequent'),[2]),
    ('impute_age',SimpleImputer(),[3]),
    ('impute_vip',SimpleImputer(strategy='most_frequent'),[4]),
    ('impute_room',SimpleImputer(),[5]),
    ('impute_food',SimpleImputer(),[6]),
    ('impute_mall',SimpleImputer(),[7]),
    ('impute_spa',SimpleImputer(),[8]),
    ('impute_vrdeck',SimpleImputer(),[9]),
    ('impute_deck',SimpleImputer(strategy='most_frequent'),[10]),
    ('impute_num',SimpleImputer(strategy='most_frequent'),[11]),
    ('impute_side',SimpleImputer(strategy='most_frequent'),[12]),
],remainder='passthrough')

In [ ]:
trf2=ColumnTransformer([
    ('encode_home',OneHotEncoder(drop='first',handle_unknown='ignore',sparse_output=True),[0]),
    ('encode_sleep',OneHotEncoder(drop='first',handle_unknown='ignore',sparse_output=True),[1]),
    ('encode_dest',OneHotEncoder(drop='first',handle_unknown='ignore',sparse_output=True),[2]),
    ('encode_vip',OneHotEncoder(drop='first',handle_unknown='ignore',sparse_output=True),[4]),
    ('encode_deck',OneHotEncoder(drop='first',handle_unknown='ignore',sparse_output=True),[10]),
    ('encode_side',OneHotEncoder(drop='first',handle_unknown='ignore',sparse_output=True),[12]),
],remainder='passthrough')

In [ ]:
X_train['RoomService'].describe()

In [ ]:
X_train.describe()

In [ ]:
for i,el in enumerate(train.columns):
    if train[el].dtype == 'float64':
        fig,axes=plt.subplots(1,2,figsize=(16,5))
        axes[0].set_title('Before Imputation')
        sns.kdeplot(ax=axes[0],data=train,x=el,fill=True)
        axes[1].set_title('After Imputation')
        sns.kdeplot(ax=axes[1],data=X_train,x=el,fill=True)

In [ ]:
pipe=Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('model',LogisticRegressionCV(max_iter=900,cv=4))
])

In [ ]:
print(np.mean(cross_val_score(pipe,X,y,cv=5,scoring='accuracy')))

In [ ]:
pipe.fit(X_train,y_train)

In [ ]:
pipe.score(X_valid,y_valid)

In [ ]:
test['Transported']=pipe.predict(test.drop(['PassengerId','Name','Cabin'],axis=1))

In [ ]:
test['Transported']=test['Transported'].map({1:True,0:False})

In [ ]:
test.sample()

In [ ]:
# test[['PassengerId','Transported']].to_csv('submission.csv',index=False)

# Using ANN

In [ ]:
X_ann=train.drop(['PassengerId','Transported','Name','Cabin'],axis=1)
y_ann=train['Transported']

In [ ]:
X_train_ann,X_test_ann,y_train_ann,y_test_ann=train_test_split(X_ann,y_ann,random_state=42)

In [ ]:
le=LabelEncoder()
y_train_ann=le.fit_transform(y_train_ann)
y_test_ann=le.transform(y_test_ann)

In [ ]:
trf1=ColumnTransformer([
    ('impute_home',SimpleImputer(strategy='most_frequent'),[0]),
    ('impute_sleep',SimpleImputer(strategy='most_frequent'),[1]),
    ('impute_dest',SimpleImputer(strategy='most_frequent'),[2]),
    ('impute_age',SimpleImputer(),[3]),
    ('impute_vip',SimpleImputer(strategy='most_frequent'),[4]),
    ('impute_room',SimpleImputer(),[5]),
    ('impute_food',SimpleImputer(),[6]),
    ('impute_mall',SimpleImputer(),[7]),
    ('impute_spa',SimpleImputer(),[8]),
    ('impute_vrdeck',SimpleImputer(),[9]),
    ('impute_deck',SimpleImputer(strategy='most_frequent'),[10]),
    ('impute_num',SimpleImputer(strategy='most_frequent'),[11]),
    ('impute_side',SimpleImputer(strategy='most_frequent'),[12]),
],remainder='passthrough')

In [ ]:
trf2=ColumnTransformer([
    ('encode_home',OneHotEncoder(drop='first',handle_unknown='ignore',sparse_output=True),[0]),
    ('encode_sleep',OneHotEncoder(drop='first',handle_unknown='ignore',sparse_output=True),[1]),
    ('encode_dest',OneHotEncoder(drop='first',handle_unknown='ignore',sparse_output=True),[2]),
    ('encode_vip',OneHotEncoder(drop='first',handle_unknown='ignore',sparse_output=True),[4]),
    ('encode_deck',OneHotEncoder(drop='first',handle_unknown='ignore',sparse_output=True),[10]),
    ('encode_side',OneHotEncoder(drop='first',handle_unknown='ignore',sparse_output=True),[12]),
],remainder='passthrough')

In [ ]:
X_train_ann=trf1.fit_transform(X_train_ann)
X_test_ann=trf1.transform(X_test_ann)

In [ ]:
X_train_ann=trf2.fit_transform(X_train_ann)
X_test_ann=trf2.transform(X_test_ann)

In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler

In [ ]:
scale=StandardScaler()

X_train_ann_scaled=scale.fit_transform(X_train_ann)
X_test_ann_scaled=scale.transform(X_test_ann)

In [ ]:
X_train_ann_scaled.shape

In [ ]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
model=Sequential([
    Dense(21,activation='relu',input_shape=[21]),
    Dropout(rate=0.3),
    Dense(15,activation='relu'),
    Dense(15,activation='relu'),
    Dropout(rate=0.3),
    Dense(7,activation='relu'),
    Dense(1,activation='sigmoid')
])

In [ ]:
early_stopping=EarlyStopping(patience=10,min_delta=0.001,restore_best_weights=True)

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['binary_accuracy'])

In [ ]:
model.summary()

In [ ]:
history=model.fit(X_train_ann_scaled,y_train_ann,epochs=200,validation_data=(X_test_ann_scaled,y_test_ann),callbacks=[early_stopping])
plt.plot(history.history['binary_accuracy'],label='Training Data')
plt.plot(history.history['val_binary_accuracy'],label='Validation Data')
plt.legend()
plt.show()

In [ ]:
print("Best Validation Accuracy:{:.4f}".format(max(history.history['val_binary_accuracy'])))

In [ ]:
test_data=trf1.transform(test.drop(['PassengerId','Name','Cabin'],axis=1))
test_data=trf2.transform(test_data)
test_data=scale.transform(test_data)
y_test_pred=model.predict(test_data)

In [ ]:
output=test['PassengerId']
Transported=np.where(y_test_pred<0.5,False,True)
output=pd.concat([output,pd.DataFrame(Transported)],axis=1)
output.rename(columns={0:'Transported'},inplace=True)
output.to_csv('submission.csv',index=False)